In [52]:
import random
import numpy as np
import scanpy as sc
import anndata as an
import scipy.stats as st
from scipy.spatial import KDTree
import math
import pandas as pd

In [53]:
counts_path = "/Users/juliafoyer/Documents/Skolarbete/Masters_thesis/her2st_G2/G2.tsv"
obs = pd.read_csv(counts_path, delimiter='\t',)
metrics_path = "/Users/juliafoyer/Documents/Skolarbete/Masters_thesis/her2st_G2/G2_selection.tsv"
metrics = pd.read_csv(metrics_path, delimiter='\t')

In [54]:
obs["Unnamed: 0"]

0      10x10
1      10x11
2      10x12
3      10x13
4      10x14
       ...  
462     9x24
463     9x25
464      9x7
465      9x8
466      9x9
Name: Unnamed: 0, Length: 467, dtype: object

In [55]:
metrics

,x,y,new_x,new_y,pixel_x,pixel_y,selected
0,18,5,17.912,5.017,4928.79,1165.28,1
1,19,5,18.924,5.022,5223.73,1166.73,1
2,20,5,19.938,5.035,5519.24,1170.51,1
3,20,6,19.958,5.998,5525.07,1449.86,1
4,16,6,15.902,5.996,4343.00,1449.28,1
...,...,...,...,...,...,...,...
462,22,26,21.866,26.003,6081.13,7253.08,1
463,19,26,18.940,26.018,5228.39,7257.43,1
464,20,26,19.928,26.038,5516.33,7263.23,1
465,23,26,23.031,26.039,6420.66,7263.52,1


In [56]:
obs.index

RangeIndex(start=0, stop=467, step=1)

In [57]:
obs

,Unnamed: 0,FO538757.1,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,...,RAB39B,CLIC2,TMLHE,SPRY3,VAMP7,USP9Y,DDX3Y,UTY,TMSB4Y,EIF1AY
0,10x10,0,0,0,0,0,0,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,10x11,0,0,0,0,0,0,1,2,1,...,0,0,0,0,0,0,0,0,0,0
2,10x12,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
3,10x13,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,10x14,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,9x24,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
463,9x25,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
464,9x7,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
465,9x8,0,0,0,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0


In [58]:
help(metrics.reindex)

Help on method reindex in module pandas.core.frame:

reindex(labels=None, index=None, columns=None, axis=None, method=None, copy=True, level=None, fill_value=nan, limit=None, tolerance=None) method of pandas.core.frame.DataFrame instance
    Conform DataFrame to new index with optional filling logic.
    
    Places NA/NaN in locations having no value in the previous index. A new object
    is produced unless the new index is equivalent to the current one and
    ``copy=False``.
    
    Parameters
    ----------
    labels : array-like, optional
                New labels / index to conform the axis specified by 'axis' to.
    index, columns : array-like, optional
        New labels / index to conform to, should be specified using
        keywords. Preferably an Index object to avoid duplicating data.
    axis : int or str, optional
                Axis to target. Can be either the axis name ('index', 'columns')
                or number (0, 1).
    method : {None, 'backfill'/'bfill',

In [59]:
xy = obs['Unnamed: 0'].tolist()
metrics["xy"] = metrics["x"].astype(str) + "x" + metrics["y"].astype(str)
metrics = metrics.set_index('xy')
metrics = metrics.reindex(index=obs['Unnamed: 0'])
metrics = metrics.reset_index()

In [60]:
metrics

,Unnamed: 0,x,y,new_x,new_y,pixel_x,pixel_y,selected
0,10x10,10,10,9.883,9.928,2588.84,2589.91,1
1,10x11,10,11,9.899,10.948,2593.50,2885.80,1
2,10x12,10,12,9.880,11.946,2587.97,3175.31,1
3,10x13,10,13,9.902,12.917,2594.38,3456.98,1
4,10x14,10,14,9.892,13.925,2591.46,3749.39,1
...,...,...,...,...,...,...,...,...
462,9x24,9,24,8.963,24.000,2320.72,6672.03,1
463,9x25,9,25,8.983,25.020,2326.55,6967.92,1
464,9x7,9,7,8.965,7.011,2321.30,1743.72,1
465,9x8,9,8,8.967,8.008,2321.88,2032.94,1


In [61]:
pix_xy = metrics.loc[:, ['pixel_x', 'pixel_y']]

In [62]:
pix_xy

,pixel_x,pixel_y
0,2588.84,2589.91
1,2593.50,2885.80
2,2587.97,3175.31
3,2594.38,3456.98
4,2591.46,3749.39
...,...,...
462,2320.72,6672.03
463,2326.55,6967.92
464,2321.30,1743.72
465,2321.88,2032.94


In [63]:
spatial = pix_xy.values

In [64]:
spatial

array([[2588.84, 2589.91],
       [2593.5 , 2885.8 ],
       [2587.97, 3175.31],
       [2594.38, 3456.98],
       [2591.46, 3749.39],
       [2592.05, 4044.12],
       [2589.13, 4337.4 ],
       [2592.05, 4624.3 ],
       [2592.63, 4915.55],
       [2596.71, 5223.91],
       [2586.8 , 5502.68],
       [2589.13, 5787.26],
       [2594.09, 6086.63],
       [2590.3 , 6374.69],
       [2597.58, 6671.74],
       [2590.3 , 6963.57],
       [2586.51, 1735.31],
       [2590.59, 2019.59],
       [2587.09, 2319.55],
       [2888.73, 2591.94],
       [2879.11, 3180.24],
       [2892.81, 3459.88],
       [2888.44, 3753.74],
       [2884.65, 4047.6 ],
       [2885.52, 4337.69],
       [2888.15, 4623.72],
       [2886.69, 4917.58],
       [2885.23, 5212.89],
       [2884.36, 5505.58],
       [2891.64, 5789.  ],
       [2894.27, 6080.54],
       [2882.61, 6381.36],
       [2891.64, 6671.45],
       [2877.07, 6970.24],
       [2877.65, 1738.21],
       [2882.03, 2019.3 ],
       [2879.99, 2309.1 ],
 

In [65]:
X = obs
del X['Unnamed: 0']

In [66]:
adata = an.AnnData(X)

/Users/juliafoyer/opt/anaconda3/envs/isbad2/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [67]:
adata.obsm['spatial'] = spatial

In [68]:
adata.obsm['spatial']

array([[2588.84, 2589.91],
       [2593.5 , 2885.8 ],
       [2587.97, 3175.31],
       [2594.38, 3456.98],
       [2591.46, 3749.39],
       [2592.05, 4044.12],
       [2589.13, 4337.4 ],
       [2592.05, 4624.3 ],
       [2592.63, 4915.55],
       [2596.71, 5223.91],
       [2586.8 , 5502.68],
       [2589.13, 5787.26],
       [2594.09, 6086.63],
       [2590.3 , 6374.69],
       [2597.58, 6671.74],
       [2590.3 , 6963.57],
       [2586.51, 1735.31],
       [2590.59, 2019.59],
       [2587.09, 2319.55],
       [2888.73, 2591.94],
       [2879.11, 3180.24],
       [2892.81, 3459.88],
       [2888.44, 3753.74],
       [2884.65, 4047.6 ],
       [2885.52, 4337.69],
       [2888.15, 4623.72],
       [2886.69, 4917.58],
       [2885.23, 5212.89],
       [2884.36, 5505.58],
       [2891.64, 5789.  ],
       [2894.27, 6080.54],
       [2882.61, 6381.36],
       [2891.64, 6671.45],
       [2877.07, 6970.24],
       [2877.65, 1738.21],
       [2882.03, 2019.3 ],
       [2879.99, 2309.1 ],
 